# Program name: PngInfo_To_Txt_ForMultiplePng_v03.ipynb

## Purpose:
1. Extract the `Positive prompt value`, `Negative prompt value`, `width`, and `height` from the png files generated by Stable Diffusion Webui.
2. Format the `Positive prompt value`, `Negative prompt value`, `width`, and `height` for using in "Prompts from file or textbox" of Stable Diffusion Webui.
3. Output the formatted text in the `*.txt` file.
4. Batch process the above steps.

## Usage 1:
1. Put the `*.ipynb` file in the target folder (where you run the `*.ipynb` file).
2. Create a folder named `PNG files` under the target folder.
3. Put the target png files in the folder `PNG files`.
4. Run all cells in the `*.ipynb` file. Or use the hot key `Shift + Enter` to run the program, since just only one cell in the program.
5. A file named `png_info_collection.txt` will be created in the target folder after the process. It can be used in "Prompts from file or textbox" of Stable Diffusion Webui.

## Usage 2:
Modify the paths of the input and output folder by yourself. It's convenient for specific users.

## Suitable version of Stable Diffusion Webui:
v1.7.0 (tested on Mar. 09, 2024)

## Variations in v3
1. **Code position adjustment.** Move the "File I/O" section to after the "Import modules" section. For users to modify the path of png folder and output filename easily.
2. **Add resolution prompts `--width` and `--height`.** Analyze the `Size` information from the image parameter. Add resolution prompts, i.e., `--width` and `--height`, to the `Output_text`.
3. **Add output path.** Add the output path for the output file in the "File I/O" section. For users to copy and paste their target path easily.
4. **Typo correction.** Modify the typos in the comments of the "File I/O" section and other places.

## Other information:
Date:   Mar. 09, 2024  
Author: Dr. Hsien-Ching Chung  
ORCiD:  https://orcid.org/0000-0001-9364-8858  
Project link: https://github.com/HsienChing/PngInfo_To_Txt_ForMultiplePng_StableDiffusionWebui  
License: MIT License  

Copyright (c) 2024 Hsien-Ching Chung  

In [1]:
#----------------------------------------
# Import modules
#----------------------------------------

from os import walk
from os.path import join
from PIL import Image


#----------------------------------------
# File I/O
#----------------------------------------

## [Input] Set the input path and foldername of the input folder.
# Default path and foldername: input_path_PngFolder = './PNG files'
# You can change the input path and foldername you like.
# Note: The png files should be generated by Stable Diffusion Webui. 
#       Otherwise, errors might occur.
input_path_PngFolder = './PNG files' 

## [Output] Set the output path and filename of the output file.
# Default path:     output_path     = './' # means current working folder
# Default filename: output_filename = 'png_info_collection.txt'
# You can change the output path and filename you like.
output_path          = './'
output_filename      = 'png_info_collection.txt'

output_path_filename = join(output_path, output_filename)
# print(output_path_filename)

# Note:
# On Windows, paths are written using backslashes (\) as the separator between folder names. 
# On Unix-based operating systems, such as macOS, Linux, and BSDs, the forward slash (/), is used as the path separator.


#----------------------------------------
# Define function()
#----------------------------------------

def PngInfoToTxt(filepath):
    ## Open image file according to filepath.
    img = Image.open(filepath)
    
    ## Read image parameters.
    # The image parameters are stored in the dictionary type variable with length 1.
    # image parameters format example: 
    #     {'parameters': '[Positive prompt value]Negative prompt: [Value]Steps: [Value], 
    #                     Sampler: [Value], CFG scale: [Value], Seed: [Value], Size: [Value], 
    #                     Model hash: [Value], Model: [Value], Denoising strength: [Value], 
    #                     Hires upscale: [Value], Hires upscaler: [Value], Lora hashes: [Value], 
    #                     TI hashes: [Value], Version: [Value]'}
    parameters_dict = img.text  
    # print(parameters_dict)       # output: {'parameters': [Parameter values]}
    # print(type(parameters_dict)) # output: <class 'dict'>
    # print( len(parameters_dict)) # output: 1

    ## Extrac string of parameter values.
    parameter_value = parameters_dict['parameters']
    # print(parameter_value)       # [Parameter values]
    # print(type(parameter_value)) # output: <class 'str'>
    # print( len(parameter_value)) # output: string lenght

    ## Extract string positions.
    Position_Negative_prompt = parameter_value.find('Negative prompt:')
    Position_Steps           = parameter_value.find('Steps:')
    Position_Size            = parameter_value.find('Size:')
    
    ## Extract the string of the positive prompt.
    Positive_prompt_original = parameter_value[:Position_Negative_prompt]
    # print(Positive_prompt_original)

    ## Eliminate '\n' in the positive prompt string.
    Positive_prompt_oneline  = Positive_prompt_original.replace('\n', '')
    # print(Positive_prompt_oneline)

    ## Extract the string of the negative prompt.
    Negative_prompt_original = parameter_value[(Position_Negative_prompt + len('Negative prompt: ')):Position_Steps]
    # print(Negative_prompt_original)

    ## Eliminate '\n' in the negative prompt string.
    Negative_prompt_oneline  = Negative_prompt_original.replace('\n', '')
    # print(Negative_prompt_oneline)

    ## Extract the string of the Size prompt.
    parameter_value_residual = parameter_value[(Position_Size + len('Size: ')):]
    Position_comma           = parameter_value_residual.find(',')
    
    Size_prompt_original     = parameter_value_residual[:Position_comma]
    # print(Size_prompt_original)

    ## Extract the strings for --width and --height.
    Position_x               = Size_prompt_original.find('x')
    Width_prompt             = Size_prompt_original[              :Position_x]
    Height_prompt            = Size_prompt_original[(Position_x+1):          ]
    # print(Width_prompt)
    # print(Height_prompt)

    ## Format the output text according to Stable Diffusion Webui.
    # REF: Use the keyword "Prompts from file or textbox" to search in the link.
    #      https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Features
    # The following parameters are supported:
    #      "sd_model", "outpath_samples", "outpath_grids", "prompt_for_display", 
    #      "prompt", "negative_prompt", "styles", "seed", "subseed_strength", "subseed", 
    #      "seed_resize_from_h", "seed_resize_from_w", "sampler_index", "sampler_name", 
    #      "batch_size", "n_iter", "steps", "cfg_scale", "width", "height", 
    #      "restore_faces", "tiling", "do_not_save_samples", "do_not_save_grid"
    # Output text format in the Output_text should be: 
    #      --prompt "[Positive prompt]" 
    #      --negative_prompt "[Negative prompt]"
    #      --width "[width value]"
    #      --height "[height value]"
    Output_text = '--prompt "'          + Positive_prompt_oneline + '" ' +\
                  '--negative_prompt "' + Negative_prompt_oneline + '" ' +\
                  '--width "'           + Width_prompt            + '" ' +\
                  '--height "'          + Height_prompt           + '" '
    # print(Output_text)

    ## Return information.
    return(Output_text)


#----------------------------------------
# Main program
#----------------------------------------

## Recursive directory traversing.
for dirpath, dirnames, filenames in walk(input_path_PngFolder):
    for f in filenames:
        fullpath = join(dirpath, f)
        # print(fullpath)

        ## Use PngInfoToTxt(filepath) function to get the output text.
        Output_text = PngInfoToTxt(fullpath)

        ## Write output text to the output file.
        with open(output_path_filename, 'a') as output_file:
            output_file.write(Output_text + '\n')